# Financial Analysis with LLMs

This notebook demonstrates how to leverage Large Language Models (LLMs) and financial data APIs to perform financial analysis. We'll be using:

1. OpenAI's GPT model for natural language processing and analysis
2. Yahoo Finance API for retrieving real-time financial data
3. LangChain for creating a seamless integration between the LLM and financial data

Throughout this notebook, we'll explore how to:

- Set up the necessary environment and dependencies
- Initialize and use an LLM for financial queries
- Connect the LLM to real-time financial data using Yahoo Finance
- Create an agent that can answer complex financial questions by combining LLM capabilities with live market data

Let's dive in and explore the intersection of AI and finance!


In [ ]:
%pip install openai python-dotenv langchain langchain-openai yfinance phidata

In [80]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path='../.env')

# Access the OPENAI_API_KEY from environment variables
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [81]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [82]:
llm.invoke("What is the stock price of Apple?")

AIMessage(content="I’m sorry, but I can't provide real-time stock prices or financial data. You can check the latest stock price of Apple (AAPL) on financial news websites, stock market apps, or through your brokerage platform.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 15, 'total_tokens': 59, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-43ddeef6-f5f4-45ff-9d7b-50f468a28051-0', usage_metadata={'input_tokens': 15, 'output_tokens': 44, 'total_tokens': 59, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

### Connecting LLMs to Financial Data

Using Yahoo Finance API

In [84]:
import yfinance as yf
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import ChatOpenAI

# Initialize the language model
llm = ChatOpenAI(temperature=0)

# Define the function to get stock price
def get_stock_price(symbol):
    ticker = yf.Ticker(symbol)
    todays_data = ticker.history(period='1d')
    return f"The current stock price of {symbol} is ${todays_data['Close'].iloc[-1]:.2f}"

# Create a tool for the agent to use
tools = [
    Tool(
        name="StockPrice",
        func=get_stock_price,
        description="Useful for getting the stock price of a company. The input should be the stock symbol of the company."
    )
]

# Initialize the agent
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Example usage
query = "What's the current stock price of Microsoft?"
response = agent.invoke(query)
print(response)



> Entering new AgentExecutor chain...
I should use the StockPrice tool to get the current stock price of Microsoft.
Action: StockPrice
Action Input: MSFT
Observation: The current stock price of MSFT is $418.16
Thought:I have the information needed to answer the question.
Final Answer: The current stock price of Microsoft is $418.16.

> Finished chain.
{'input': "What's the current stock price of Microsoft?", 'output': 'The current stock price of Microsoft is $418.16.'}


In [87]:
from langchain_core.tools import tool, StructuredTool
from datetime import date

@tool
def get_stock_price(symbol):
    """Use this tool to get the stock price of a company. The input should be the stock symbol of the company."""
    ticker = yf.Ticker(symbol)
    todays_data = ticker.history(period='1d')
    return f"The current stock price of {symbol} is ${todays_data['Close'].iloc[-1]:.2f}"

In [88]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain import hub

tools = [get_stock_price]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Try to answer user query using available tools.",
        ),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm = ChatOpenAI(model = "gpt-4o-mini")

finance_agent = create_tool_calling_agent(llm, tools, prompt)

finance_agent_executor = AgentExecutor(agent=finance_agent, tools=tools, verbose=True)

In [89]:
from langchain_core.messages import HumanMessage

response = finance_agent_executor.invoke({"messages": [HumanMessage(content="What is the stock price of Apple?")]})
response




> Entering new AgentExecutor chain...

Invoking: `get_stock_price` with `{'symbol': 'AAPL'}`


The current stock price of AAPL is $235.00The current stock price of Apple (AAPL) is $235.00.

> Finished chain.


{'messages': [HumanMessage(content='What is the stock price of Apple?', additional_kwargs={}, response_metadata={})],
 'output': 'The current stock price of Apple (AAPL) is $235.00.'}

In [90]:
response = finance_agent_executor.invoke({"messages": [HumanMessage(content="What is the last dividend date of Apple?")]})
response



> Entering new AgentExecutor chain...
I currently don't have access to a tool that provides specific dividend dates for companies. However, you can usually find the last dividend date of Apple (AAPL) on financial news websites, stock market apps, or by checking Apple's investor relations website. If you need help with something else, feel free to ask!

> Finished chain.


{'messages': [HumanMessage(content='What is the last dividend date of Apple?', additional_kwargs={}, response_metadata={})],
 'output': "I currently don't have access to a tool that provides specific dividend dates for companies. However, you can usually find the last dividend date of Apple (AAPL) on financial news websites, stock market apps, or by checking Apple's investor relations website. If you need help with something else, feel free to ask!"}

Adding Stock Market Analysis Tools

In [91]:
from langchain_core.tools import tool, StructuredTool
from datetime import date

@tool
def company_information(ticker: str) -> dict:
    """Use this tool to retrieve company information like address, industry, sector, company officers, business summary, website,
       marketCap, current price, ebitda, total debt, total revenue, debt-to-equity, etc."""
    
    ticker_obj = yf.Ticker(ticker)
    ticker_info = ticker_obj.get_info()

    return ticker_info

@tool
def last_dividend_and_earnings_date(ticker: str) -> dict:
    """
    Use this tool to retrieve company's last dividend date and earnings release dates.
    It does not provide information about historical dividend yields.
    """
    ticker_obj = yf.Ticker(ticker)
    
    return ticker_obj.get_calendar()

@tool
def summary_of_mutual_fund_holders(ticker: str) -> dict:
    """
    Use this tool to retrieve company's top mutual fund holders. 
    It also returns their percentage of share, stock count and value of holdings.
    """
    ticker_obj = yf.Ticker(ticker)
    mf_holders = ticker_obj.get_mutualfund_holders()
    
    return mf_holders.to_dict(orient="records")

@tool
def summary_of_institutional_holders(ticker: str) -> dict:
    """
    Use this tool to retrieve company's top institutional holders. 
    It also returns their percentage of share, stock count and value of holdings.
    """
    ticker_obj = yf.Ticker(ticker)   
    inst_holders = ticker_obj.get_institutional_holders()
    
    return inst_holders.to_dict(orient="records")

@tool
def stock_grade_updrages_downgrades(ticker: str) -> dict:
    """
    Use this to retrieve grade ratings upgrades and downgrades details of particular stock.
    It'll provide name of firms along with 'To Grade' and 'From Grade' details. Grade date is also provided.
    """
    ticker_obj = yf.Ticker(ticker)
    
    curr_year = date.today().year
    
    upgrades_downgrades = ticker_obj.get_upgrades_downgrades()
    upgrades_downgrades = upgrades_downgrades.loc[upgrades_downgrades.index > f"{curr_year}-01-01"]
    upgrades_downgrades = upgrades_downgrades[upgrades_downgrades["Action"].isin(["up", "down"])]
    
    return upgrades_downgrades.to_dict(orient="records")

@tool
def stock_splits_history(ticker: str) -> dict:
    """
    Use this tool to retrieve company's historical stock splits data.
    """
    ticker_obj = yf.Ticker(ticker)
    hist_splits = ticker_obj.get_splits()
    
    return hist_splits.to_dict()

@tool
def stock_news(ticker: str) -> dict:
    """
    Use this to retrieve latest news articles discussing particular stock ticker.
    """
    ticker_obj = yf.Ticker(ticker)
    
    return ticker_obj.get_news()

In [92]:
tools = [
    company_information,
    last_dividend_and_earnings_date,
    stock_splits_history,
    summary_of_mutual_fund_holders,
    summary_of_institutional_holders, 
    stock_grade_updrages_downgrades,
    stock_news,
    get_stock_price
]


finance_agent = create_tool_calling_agent(llm, tools, prompt)

finance_agent_executor = AgentExecutor(agent=finance_agent, tools=tools, verbose=True)

In [93]:
response = finance_agent_executor.invoke({"messages": [HumanMessage(content="What is the last dividend date of Apple?")]})
response



> Entering new AgentExecutor chain...

Invoking: `last_dividend_and_earnings_date` with `{'ticker': 'AAPL'}`


{'Dividend Date': datetime.date(2024, 8, 15), 'Ex-Dividend Date': datetime.date(2024, 8, 12), 'Earnings Date': [datetime.date(2024, 11, 1)], 'Earnings High': 1.65, 'Earnings Low': 0.92, 'Earnings Average': 1.55, 'Revenue High': 97945300000, 'Revenue Low': 93653000000, 'Revenue Average': 94391500000}The last dividend date for Apple (AAPL) is August 15, 2024, with an ex-dividend date of August 12, 2024.

> Finished chain.


{'messages': [HumanMessage(content='What is the last dividend date of Apple?', additional_kwargs={}, response_metadata={})],
 'output': 'The last dividend date for Apple (AAPL) is August 15, 2024, with an ex-dividend date of August 12, 2024.'}

In [95]:
response = finance_agent_executor.invoke({"messages": [HumanMessage(content="What is the stock price of Reliance Industries?")]})
response



> Entering new AgentExecutor chain...

Invoking: `get_stock_price` with `{'symbol': 'RELIANCE'}`




$RELIANCE: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


IndexError: single positional indexer is out-of-bounds

In [94]:
response = finance_agent_executor.invoke({"messages": [HumanMessage(content="What are ebitda, total debt, total revenue and debt-to-equity for Nvidia stock?")]})
response



> Entering new AgentExecutor chain...

Invoking: `company_information` with `{'ticker': 'NVDA'}`


{'address1': '2788 San Tomas Expressway', 'city': 'Santa Clara', 'state': 'CA', 'zip': '95051', 'country': 'United States', 'phone': '408 486 2000', 'website': 'https://www.nvidia.com', 'industry': 'Semiconductors', 'industryKey': 'semiconductors', 'industryDisp': 'Semiconductors', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': "NVIDIA Corporation provides graphics and compute and networking solutions in the United States, Taiwan, China, Hong Kong, and internationally. The Graphics segment offers GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure, and solutions for gaming platforms; Quadro/NVIDIA RTX GPUs for enterprise workstation graphics; virtual GPU or vGPU software for cloud-based visual and virtual computing; automotive platforms for infotainment systems; and Omniverse software f

{'messages': [HumanMessage(content='What are ebitda, total debt, total revenue and debt-to-equity for Nvidia stock?', additional_kwargs={}, response_metadata={})],
 'output': 'For Nvidia (NVDA), the financial metrics are as follows:\n\n- **EBITDA**: $61,184,000,000\n- **Total Debt**: $10,014,999,552\n- **Total Revenue**: $96,307,003,392\n- **Debt-to-Equity Ratio**: 17.221\n\nIf you need further information or analysis, feel free to ask!'}

### Portfolio Analysis

In [96]:
# Create a dummy portfolio of 5 stocks
portfolio = {
    'AAPL': 50,  # Apple
    'MSFT': 30,  # Microsoft
    'GOOGL': 20, # Alphabet (Google)
    'AMZN': 15,  # Amazon
    'NVDA': 25   # NVIDIA
}

# Function to analyze a single stock
def analyze_stock(ticker: str, shares: int):
    response = finance_agent_executor.invoke({
        "messages": [
            HumanMessage(content=f"Provide a brief analysis of {ticker} including current price, P/E ratio, and recent performance.")
        ]
    })
    return f"{ticker} ({shares} shares): {response['output']}"

# Function to analyze the entire portfolio
def analyze_portfolio(portfolio: dict):
    analysis = []
    total_value = 0

    for ticker, shares in portfolio.items():
        stock_analysis = analyze_stock(ticker, shares)
        analysis.append(stock_analysis)

        # Get current price to calculate portfolio value
        price_response = finance_agent_executor.invoke({
            "messages": [
                HumanMessage(content=f"What is the current stock price of {ticker}?")
            ]
        })
        try:
            price = float(price_response['output'].split('$')[1].split()[0])
            total_value += price * shares
        except:
            print(f"Could not parse price for {ticker}")

    return analysis, total_value

# Analyze the portfolio
print("Analyzing portfolio...")
stock_analyses, portfolio_value = analyze_portfolio(portfolio)

# Print the results
print("\nPortfolio Analysis:")
for analysis in stock_analyses:
    print(analysis)
    print("-" * 50)


Analyzing portfolio...


> Entering new AgentExecutor chain...

Invoking: `get_stock_price` with `{'symbol': 'AAPL'}`


The current stock price of AAPL is $235.00
Invoking: `company_information` with `{'ticker': 'AAPL'}`


{'address1': 'One Apple Park Way', 'city': 'Cupertino', 'state': 'CA', 'zip': '95014', 'country': 'United States', 'phone': '408 996 1010', 'website': 'https://www.apple.com', 'industry': 'Consumer Electronics', 'industryKey': 'consumer-electronics', 'industryDisp': 'Consumer Electronics', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCa

### Investment Analyst

In [ ]:
%pip install phidata duckduckgo-search

In [97]:
from phi.agent import Agent
from phi.model.openai import OpenAIChat
from phi.tools.duckduckgo import DuckDuckGo
from phi.tools.yfinance import YFinanceTools

web_agent = Agent(
    name="Web Agent",
    role="Search the web for information",
    model=OpenAIChat(id="gpt-4o-mini"),
    tools=[DuckDuckGo()],
    markdown=True,
    show_tool_calls=True,
)

finance_agent = Agent(
    name="Finance Agent",
    role="Get financial data",
    model=OpenAIChat(id="gpt-4o-mini"),
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True, company_news=True)],
    instructions=["Always use tables to display data"],
    markdown=True,
    show_tool_calls=True,
)




In [100]:
agent_team.print_response("How much did Nvidia stock increase in last 1 year?", stream=True)

Output()

In [99]:
agent_team.print_response("How much did Nvidia stock increase in next 1 year?", stream=True)

Output()

In [101]:
agent_team = Agent(
    team=[web_agent, finance_agent],
    show_tool_calls=True,
    markdown=True,
)
agent_team.print_response("Research the web for NVDA and share analyst recommendations", stream=True)

Output()

### Comparing Stocks

In [104]:
def compare_stocks(agent, symbol1, symbol2, period='1y'):
    query = f"Compare the performance metrics of {symbol1} and {symbol2} over the last {period}. Do a detailed analysis on P/E ratio"
    response = agent_team.print_response(
        query,
        stream=True,
        show_full_reasoning=True
    )
    return response

In [105]:
# Compare two stocks
compare_stocks(finance_agent, "NVDA", "TSLA", "1y")

Output()

### Portfolio Recommendations

In [106]:
# Create a dummy portfolio as a string
portfolio_details = """
Portfolio:
1. AAPL (Apple Inc.): 100 shares
2. MSFT (Microsoft Corporation): 50 shares
3. GOOGL (Alphabet Inc.): 30 shares
4. AMZN (Amazon.com, Inc.): 20 shares

Please analyze this portfolio and provide:
1. A brief overview of each stock's recent performance
2. An analysis of the portfolio's diversification
3. Recommendations for potential changes or additions to the portfolio
4. Any relevant market trends that might impact this portfolio

I am ready to take only moderate risk. I want to exit the portfolio in 5 years. Tune your suggestions accordingly. 

"""

# Use the existing agent_team to analyze the portfolio
agent_team.print_response(portfolio_details, stream=True)



Output()